In [1]:
def load_hdf_group(data_dir, hdf_filename, group="/"):
    '''
    Loads any datasets from the given hdf group into a dictionary. Also will
    recursively load other groups if any exist under the given group

    Args:
        data_dir (str): folder where data is located
        hdf_filename (str): name of hdf file
        group (str): name of the group to load
    
    Returns:
        dict: all the datasets contained in the given group
    '''
    full_file_name = os.path.join(data_dir, hdf_filename)
    hdf = h5py.File(full_file_name, 'r')
    if group not in hdf:
        raise ValueError('No such group in file {}'.format(hdf_filename))

    # Recursively load groups until datasets are reached
    def _load_hdf_group(hdf):
        keys = hdf.keys()
        data = dict()
        for k in keys:
            if isinstance(hdf[k], h5py.Group):
                data[k] = _load_hdf_group(hdf[k])
            else:
                k_, v = _load_hdf_dataset(hdf[k], k)
                data[k_] = v
        return data

    data = _load_hdf_group(hdf[group])
    hdf.close()
    return data

def _load_hdf_dataset(dataset, name):
    '''
    Internal function for loading hdf datasets. Decodes json and unicode data automatically.

    Args:
        dataset (hdf object): dataset to load
        name (str): name of the dataset

    Returns:
        tuple: Tuple containing:
            | **name (str):** name of the dataset (might be modified)
            | **data (object):** loaded data
    '''
    data = dataset[()]
    if '_json' in name:
        import json
        name = name.replace('_json', '')
        data = json.loads(data)
    try:
        data = data.decode('utf-8')
    except:
        pass
    return name, data

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import aopy
import os
import pandas as pds
from db import dbfunctions as db
from ipywidgets import interactive, widgets
import scipy
import h5py
from tqdm.auto import tqdm 
import seaborn as sn
import sklearn
from sklearn.decomposition import PCA, FactorAnalysis
from itertools import compress
import multiprocessing as mp
import time
import math
from scipy.fft import fft
import glob
from datetime import date

/home/aolab/miniconda3/envs/np_targeting/lib/python3.9/site-packages/one/alf/files.py:10: FutureWarning: `one.alf.files` will be removed in version 3.0. Use `one.alf.path` instead.
  warnings.warn(


# Set parameters

In [3]:
save_figs = False
base_save_dir = "/media/moor-data/results/Ryan/neuropixel_targeting/"
np_preproc_data_folder = 'np_analysis_preproc_data'
ecog_dec_acc_file_name = 'ecog_decoding_maps/npinsert_ecog_decoding'

subject = 'affi'
align_events = ['TARGET ONSET', 'GO CUE', 'MOVEMENT ONSET']

In [4]:
# Decoding calculation parameters
tbefore = 0.5
tafter = 1
nlda_lags = 1
niter_match = 50
min_trial_prop = .85
ntrial_bin_size = 96
nfolds = 4

# Visualization parameters
colors = sn.color_palette(n_colors=9)
recording_brain_areas={'M1': [30, 56, 47, 40, 121, 48, 120, 98], 'PM':[11, 9, 18, 22, 10, 45]}
day_colors = ['dodgerblue', 'indigo', 'violet', 'lightblue', 'mediumorchid',
              'purple', 'steelblue', 'dodgerblue', 'lightblue', 'red', 'black', 'green', 'purple', 'cyan', 'gray', 'yellow'] 

save_dir = "/media/moor-data/results/Ryan/neuropixel_targeting/np_analysis_preproc_data"

# Load and extract relevant data

In [5]:
start = time.time()
aopy.utils.release_memory_limit()
df, rasters, preproc_metadata = aopy.data.base.pkl_read(f"{subject}_np_preprocessed", os.path.join(base_save_dir, np_preproc_data_folder))
print(f"{np.round((time.time()-start)/60)} min to load preprocessed data")
nrecs = preproc_metadata['nrecs']
recording_site = preproc_metadata['recording_sites'] # will be the same for all align events
implants = ['NP_Insert72' if preproc_metadata['implant'][irec] == 'NP_Insert72' else 'NP_Insert137' for irec in range(len(preproc_metadata['implant']))] #Rename because name in bmi3d is slightly different (TODO)
dates = np.unique(df['date'])

7.0 min to load preprocessed data


In [132]:
ecog_dec_acc = load_hdf_group(base_save_dir, ecog_dec_acc_file_name)

In [133]:
qc_results, ksdrift = aopy.data.base.pkl_read(f"{subject}_QCunits", os.path.join(base_save_dir, np_preproc_data_folder))
# stable_unit_labels = [qc_results['final_good_unit_labels'][irec] for irec in range(nrecs)]
# stable_unit_idx = [qc_results['final_good_unit_idx'][irec] for irec in range(nrecs)]
# nstable_unit = np.array([len(qc_results['final_good_unit_idx'][irec]) for irec in range(nrecs)])
# neuron_pos = [qc_results['position'][irec] for irec in range(nrecs)]

# if subject == 'affi':
stable_unit_labels = [qc_results['manual_good_unit_labels'][irec] for irec in range(nrecs)]
stable_unit_idx = [qc_results['manual_good_unit_idx'][irec] for irec in range(nrecs)]
nstable_unit = np.array([len(qc_results['manual_good_unit_idx'][irec]) for irec in range(nrecs)])
neuron_pos = [qc_results['manual_position'][irec] for irec in range(nrecs)]

In [134]:
if subject == 'beignet':
    surface_pos = np.ones(nrecs)*3840
elif subject == 'affi':
    surface_pos = np.array([3840, 3840, 3400, 3250,3840,3800,3800,3900,3500,3700,2500,3250,2100,3840,3100,1600,3250,2500,3100,3300,3800,3100,3250,2750,2900, 2750,
                           3000,3000, 3000,3700, 3000,3700, 3000,3700, 35000])
print(preproc_metadata['recording_sites'])
print(surface_pos.shape, nrecs, surface_pos)

[ 19  29  55  58  33  98  10  74  72 107  70  44  98  31  71  21  69  95
  31  31  70  31  72  72  73  57  65  10  64  17  58  17  58  18  49]
(35,) 35 [ 3840  3840  3400  3250  3840  3800  3800  3900  3500  3700  2500  3250
  2100  3840  3100  1600  3250  2500  3100  3300  3800  3100  3250  2750
  2900  2750  3000  3000  3000  3700  3000  3700  3000  3700 35000]


In [135]:
# compile dataframe of unit information
# Need columns for: date, recording site, unit_label, absolute y-pos, relative y-pos, waveform
unit_info = {'rec_number': [], 'date': [], 'rec_site': [], 'rec_xpos': [], 'rec_ypos': [], 'rec_rcaxis': [], 'rec_mlaxis': [], 'implant': [],
             'unit_label': [], 'unit_idx': [], 'abs_depth': [], 'rel_depth': [], 'waveform': [], 'surface_pos': [], 'penetration': []}
# unit_info = {'rec_number': [], 'date': [], 'rec_site': [], 'rec_xpos': [], 'rec_ypos': [], 'implant': [],
#              'unit_label': [], 'unit_idx': []}
surface_buffer = 200
for irec in tqdm(range(nrecs)):
    # print(irec, nstable_unit[irec], neuron_pos[irec].shape)
    for iunit in range(nstable_unit[irec]):
        # Only save units that are below the estimated surface of the brain
        # if neuron_pos[irec][iunit] <= (surface_pos[irec]+surface_buffer):   
        
        unit_info['rec_number'].append(irec) # Recording_number
        unit_info['date'].append(list(df[df['penetration']==irec]['date'])[0]) # date (assumes each recording is done on a different day)
        unit_info['rec_site'].append(preproc_metadata['recording_sites'][irec]) # Recording site
        unit_info['rec_xpos'].append(ecog_dec_acc[subject]['rec_locations'][irec,0]) # Recording site x pos in chamber
        unit_info['rec_ypos'].append(ecog_dec_acc[subject]['rec_locations'][irec,1]) # Recording site y pos in chamber
        unit_info['rec_rcaxis'].append(ecog_dec_acc[subject]['rc_axis'][irec,0]) # Recording site y pos in chamber
        unit_info['rec_mlaxis'].append(ecog_dec_acc[subject]['rc_axis'][irec,1]) # Recording site y pos in chamber
        unit_info['implant'].append(implants[irec]) # Implant
        unit_info['unit_label'].append(stable_unit_labels[irec][iunit]) # unit label output from kilosort
        unit_info['unit_idx'].append(stable_unit_idx[irec][iunit]) # unit label output from kilosort
        unit_info['abs_depth'].append(surface_pos[irec] - neuron_pos[irec][iunit]) #Absolute depth [um]
        unit_info['rel_depth'].append(3840-neuron_pos[irec][iunit]-np.min(3840-neuron_pos[irec])) #Relative depth [um] (to top detected neuron)
        unit_info['waveform'].append(qc_results['mean_wfs'][irec][iunit,:]) #waveform
        unit_info['surface_pos'].append(surface_pos[irec])
        unit_info['penetration'].append(irec)        

  0%|          | 0/35 [00:00<?, ?it/s]

In [136]:
unit_df = pds.DataFrame(unit_info)

# Create pseudopopulations

In [137]:
pp_configs = ['random', 'column', 'depth'] # Pseudopopulation cofigurations 

## Random

In [138]:
# Create random neuron populations
nrandom_units = 10 # Number of random units chosen in each group
nrandom_groups = 1000 # Number of random groups to make
random_group_info = []
for igroup in range(nrandom_groups):
    group_idx = np.random.choice(np.arange(len(unit_df)), size=(nrandom_units), replace=False)
    random_group_info.append(unit_df.loc[group_idx])

## Column

In [139]:
unique_rec_pos, unique_rec_pos_idx = np.unique(ecog_dec_acc[subject]['rec_locations'], axis=0, return_index=True)
column_group_info = []

In [140]:
# Create random neuron populations from each recording_location
min_probe_depth = 0 #[um]
unique_rec_pos, unique_rec_pos_idx = np.unique(ecog_dec_acc[subject]['rec_locations'], axis=0, return_index=True)
column_group_info = []
for ipos in range(unique_rec_pos.shape[0]):
    pos = unique_rec_pos[ipos,:]
    new_df = unit_df.loc[(unit_df['rec_xpos']==pos[0]) & (unit_df['rec_ypos']==pos[1]) & (unit_df['surface_pos']>min_probe_depth)].reset_index()
    
    # Add a row of 0 if there are no units so code works. There is no recording site 0 because it is 1-indexed.
    if len(new_df)==0:   
        new_df.loc[0] = [np.nan] * len(new_df.columns)
        new_df['rec_site'] = ecog_dec_acc[subject]['rec_sites'][unique_rec_pos_idx[ipos]]
        new_df['implant'] = implants[unique_rec_pos_idx[ipos]]
        
    column_group_info.append(new_df) #only use sites whose surface position is greater than a threshold

    # print(unit_df.loc[(unit_df['rec_xpos']==pos[0]) & (unit_df['rec_ypos']==pos[1])].reset_index())
    # print(len(column_group_info[ipos]),column_group_info[ipos]['rec_site'][0])

In [141]:
new_df

,index,rec_number,date,rec_site,rec_xpos,rec_ypos,rec_rcaxis,rec_mlaxis,implant,unit_label,unit_idx,abs_depth,rel_depth,waveform,surface_pos,penetration
0,1226,32,2024-09-23,58,3.99999,0.0,1.999995,-3.464093,NP_Insert72,52,52,2220.0,700.0,"[-4.378369030522715, -4.4120425320692975, -4.3...",3000,32
1,1227,32,2024-09-23,58,3.99999,0.0,1.999995,-3.464093,NP_Insert72,57,57,2200.0,680.0,"[-4.551386602243701, -4.817894933153141, -4.48...",3000,32
2,1228,32,2024-09-23,58,3.99999,0.0,1.999995,-3.464093,NP_Insert72,65,65,2020.0,500.0,"[-4.019124145408933, -4.589423775303875, -4.38...",3000,32
3,1229,32,2024-09-23,58,3.99999,0.0,1.999995,-3.464093,NP_Insert72,78,78,1840.0,320.0,"[-5.12319391243929, -4.927919032825895, -4.984...",3000,32
4,1230,32,2024-09-23,58,3.99999,0.0,1.999995,-3.464093,NP_Insert72,86,86,1700.0,180.0,"[-5.391229963986109, -5.499494145658969, -5.74...",3000,32
5,1231,32,2024-09-23,58,3.99999,0.0,1.999995,-3.464093,NP_Insert72,98,98,1540.0,20.0,"[-3.0771761656215495, -3.580490441940367, -4.0...",3000,32
6,1232,32,2024-09-23,58,3.99999,0.0,1.999995,-3.464093,NP_Insert72,100,100,1520.0,0.0,"[1.6295806279497975, 1.446419606971006, 1.5917...",3000,32


In [142]:
print('nUnits:  ', [len(column_group_info[ii]) for ii in range(len(column_group_info))])
print('Rec Site:', [column_group_info[ii]['rec_site'][0] for ii in range(len(column_group_info))])
print(len(unique_rec_pos))
print(unique_rec_pos)
print(recording_site)

nUnits:   [49, 72, 44, 13, 78, 41, 18, 1, 43, 21, 34, 29, 72, 22, 45, 46, 72, 152, 48, 16, 1, 38, 183, 30, 60, 7]
Rec Site: [29, 19, 10, 17, 10, 31, 21, 18, 55, 44, 33, 69, 70, 58, 95, 71, 107, 72, 64, 57, 48, 73, 98, 74, 65, 58]
26
[[-3.79999184  0.75999837]
 [-3.79999184  2.2799951 ]
 [-3.79999184  3.79999184]
 [-2.99999244  0.99999748]
 [-2.99999244  2.99999244]
 [-2.2799951   2.2799951 ]
 [-2.2799951   3.79999184]
 [-1.99999496  1.99999496]
 [-1.51999674  0.        ]
 [-0.75999837  2.2799951 ]
 [-0.75999837  3.79999184]
 [ 0.          0.        ]
 [ 0.75999837  0.75999837]
 [ 0.75999837  2.2799951 ]
 [ 1.51999674 -1.51999674]
 [ 1.51999674  1.51999674]
 [ 2.2799951  -2.2799951 ]
 [ 2.2799951   2.2799951 ]
 [ 2.99999244 -2.99999244]
 [ 2.99999244 -0.99999748]
 [ 2.99999244  0.99999748]
 [ 3.03999347  3.03999347]
 [ 3.79999184  0.75999837]
 [ 3.79999184  3.79999184]
 [ 3.99998992 -1.99999496]
 [ 3.99998992  0.        ]]
[ 19  29  55  58  33  98  10  74  72 107  70  44  98  31  71  21

## Depth

In [143]:
# depth_ranges = [(0,1000), (1000, 2000), (2000, 100000)] #Boundary distinguishing shallow and deep neurons 

In [144]:
# Create random neuron populations from neurons at different depths

depth_ranges = [(0,1000),(250,1250),(500,1500),(750,1750),(1000,2000),(1250,2250),(1500,2500),(1750,2750),(2000,3000),(2250,3250),(2500,3500),(2750,3750),(np.max(unit_df['rel_depth'])-1000,np.max(unit_df['rel_depth']))] #Boundary distinguishing shallow and deep neurons 

depth_group_info = []
for irange, depth_range in enumerate(depth_ranges):
    depth_group_info.append(unit_df.loc[(unit_df['rel_depth'] >= depth_range[0]) & (unit_df['rel_depth'] < depth_range[1])].reset_index())

In [145]:
#Create random neuron populations for each recording site from neurons at different depths

depth_group_info_by_site = {}
unique_sites = np.unique(unit_df['rec_site'])
for isite, site in enumerate(unique_sites):
    depth_group_info_by_site[site] = []
    for irange, depth_range in enumerate(depth_ranges):
        depth_mask = (unit_df['rel_depth'] >= depth_range[0]) & (unit_df['rel_depth'] < depth_range[1])
        site_mask = unit_df['rec_site'] == site
        depth_group_info_by_site[site].append(unit_df.loc[depth_mask*site_mask].reset_index())

In [146]:
for isite, site in enumerate(unique_sites):
    print(site, [len(depth_group_info_by_site[site][idepth]) for idepth in range(len(depth_group_info_by_site[site]))])

10 [50, 44, 44, 36, 28, 23, 14, 15, 26, 30, 31, 31, 32]
17 [10, 5, 2, 0, 0, 0, 2, 2, 2, 3, 1, 1, 1]
18 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
19 [21, 27, 25, 30, 25, 25, 27, 22, 24, 17, 13, 7, 7]
21 [8, 8, 6, 7, 6, 5, 7, 6, 4, 3, 0, 0, 0]
29 [15, 15, 22, 25, 25, 24, 20, 15, 9, 6, 3, 1, 1]
31 [24, 17, 17, 16, 14, 13, 10, 6, 3, 1, 0, 0, 0]
33 [7, 8, 14, 18, 20, 18, 14, 11, 7, 4, 2, 0, 0]
44 [4, 6, 10, 11, 10, 10, 7, 7, 7, 4, 3, 1, 1]
55 [16, 17, 15, 12, 6, 4, 5, 5, 6, 12, 13, 16, 16]
57 [5, 5, 8, 9, 11, 9, 5, 2, 0, 0, 0, 0, 0]
58 [27, 19, 12, 4, 2, 1, 0, 0, 0, 0, 0, 0, 0]
64 [8, 14, 14, 13, 17, 16, 17, 20, 20, 16, 14, 8, 7]
65 [5, 6, 13, 21, 30, 34, 37, 32, 25, 18, 7, 3, 3]
69 [20, 20, 17, 14, 9, 4, 2, 0, 0, 0, 0, 0, 0]
70 [32, 32, 25, 21, 18, 18, 22, 23, 22, 15, 9, 3, 3]
71 [4, 5, 9, 17, 28, 36, 35, 26, 14, 4, 0, 0, 0]
72 [30, 27, 31, 35, 40, 53, 52, 48, 57, 49, 50, 43, 41]
73 [6, 12, 14, 26, 31, 24, 21, 7, 1, 1, 0, 0, 0]
74 [4, 4, 7, 9, 10, 9, 5, 7, 8, 16, 16, 12, 11]
95 [7, 6, 11, 12

In [147]:
print(column_group_info[-4])

     index  rec_number        date  rec_site  rec_xpos  rec_ypos  rec_rcaxis  \
0      220           5  2024-06-03        98  3.799992  0.759998    2.558174   
1      221           5  2024-06-03        98  3.799992  0.759998    2.558174   
2      222           5  2024-06-03        98  3.799992  0.759998    2.558174   
3      223           5  2024-06-03        98  3.799992  0.759998    2.558174   
4      224           5  2024-06-03        98  3.799992  0.759998    2.558174   
..     ...         ...         ...       ...       ...       ...         ...   
178    684          12  2024-08-30        98  3.799992  0.759998    2.558174   
179    685          12  2024-08-30        98  3.799992  0.759998    2.558174   
180    686          12  2024-08-30        98  3.799992  0.759998    2.558174   
181    687          12  2024-08-30        98  3.799992  0.759998    2.558174   
182    688          12  2024-08-30        98  3.799992  0.759998    2.558174   

     rec_mlaxis       implant  unit_lab

# Save

In [148]:
pseudopopulation_metadata = {'nrandom_units': nrandom_units, 'nrandom_groups': nrandom_groups, 'depth_ranges': depth_ranges}
print(pseudopopulation_metadata)

{'nrandom_units': 10, 'nrandom_groups': 1000, 'depth_ranges': [(0, 1000), (250, 1250), (500, 1500), (750, 1750), (1000, 2000), (1250, 2250), (1500, 2500), (1750, 2750), (2000, 3000), (2250, 3250), (2500, 3500), (2750, 3750), (2780.0, 3780.0)]}


In [149]:
# Save data
aopy.data.base.pkl_write(f"{subject}_np_psuedopopulations", (random_group_info, column_group_info, depth_group_info, depth_group_info_by_site, pseudopopulation_metadata, unit_df), save_dir)

In [150]:
print(len(depth_group_info[0]))
print(len(depth_group_info[1]))
print(len(depth_group_info[2]))


343
347
376


In [151]:
aopy.utils.get_memory_available_gb()
type(df['unit_labels'][df['date']==dates[0]][0])

numpy.ndarray